In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict

df_old_1 = pd.DataFrame()
df_old = pd.read_csv('State_1_Crash_Details.csv')
df_old['Weather'].replace({'Cloudy':'Clear','Unknown':'Not Reported','Other':'Not Reported','Sleet, Hail (Freezing Rain or Drizzle)':'Rain','Freezing Rain or Drizzle':'Rain'}, inplace = True)
df_old['Light_Type'].replace({'Dark - Not Lighted':'Dark','Dark - Lighted': 'Dark','Dark - Unknown Lighting':'Dark','Not Reported':'Dark','Other':'Dark','Unknown':'Dark','Dawn':'Dawn/Dusk','Dusk':'Dawn/Dusk'}, inplace = True)
df_old.dropna(inplace = True)
ohe = OneHotEncoder(handle_unknown = 'ignore')
encoder_df = pd.DataFrame(ohe.fit_transform(df_old[['Weather']]).toarray())
encoder_df1 = pd.DataFrame(ohe.fit_transform(df_old[['Light_Type']]).toarray())
encoder_df.rename(columns = {0 : 'Clear', 1 : 'Fog, Smog, Smoke',2: 'Weather Not Reported',3: 'Rain',4: 'Snow'}, inplace = True)
encoder_df1.rename(columns = {0 : 'Dark', 1 : 'Dawn/Dusk',2: 'Daylight'}, inplace = True)
final_df = pd.concat([df_old.dropna(),encoder_df.dropna(), encoder_df1.dropna()], axis = "columns" )

final_df.dropna(inplace = True)
final_df.drop(final_df.columns[[0,2,3]], axis = 1, inplace = True)
print(final_df)

final_df.to_csv('Test.csv')

     State Case  Clear  Fog, Smog, Smoke  Weather Not Reported  Rain  Snow  \
0       10002.0    1.0               0.0                   0.0   0.0   0.0   
1       10003.0    1.0               0.0                   0.0   0.0   0.0   
2       10004.0    0.0               0.0                   0.0   1.0   0.0   
3       10005.0    1.0               0.0                   0.0   0.0   0.0   
4       10006.0    1.0               0.0                   0.0   0.0   0.0   
..          ...    ...               ...                   ...   ...   ...   
741     10743.0    1.0               0.0                   0.0   0.0   0.0   
742     10744.0    1.0               0.0                   0.0   0.0   0.0   
743     10745.0    1.0               0.0                   0.0   0.0   0.0   
744     10746.0    1.0               0.0                   0.0   0.0   0.0   
745     10747.0    0.0               0.0                   1.0   0.0   0.0   

     Dark  Dawn/Dusk  Daylight  
0     0.0        0.0       1.0

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from collections import defaultdict
final_dataframe1 = pd.DataFrame(columns = [
        "State Case",
        "Weather",
        "Light_Type"
    ])

def crash_details(state_Case, year, state_id):
    data_list = list()
    url = f'https://crashviewer.nhtsa.dot.gov/CrashAPI//crashes/GetCaseDetails?stateCase={state_Case}&caseYear={year}&state={state_id}&format=json'
    page = requests.get(url)
    data = page.text
    parse_json = json.loads(data)
    argument_crash = defaultdict(list)
    try:
        data_list.append(parse_json['Results'][0][0]['CrashResultSet']['ST_CASE'])
    except:
        data_list.append(None)
    try:
        data_list.append(parse_json['Results'][0][0]['CrashResultSet']['WEATHER1NAME'])
    except:
        data_list.append(None)
    try:
        data_list.append(parse_json['Results'][0][0]['CrashResultSet']['LGT_CONDNAME'])
    except:
        data_list.append(None)
    return data_list


case_id = 10002
case_id_end = 10812
year = 2014
state = 1
for i in range(case_id,case_id_end,1):
   data_list = crash_details(i,year,state)
   final_dataframe1.loc[len(final_dataframe1)] = data_list
#    print(final_dataframe1)
# final_dataframe1.to_csv('State_1_Crash_Details.csv')


final_dataframe1['Weather'].replace({'Cloudy':'Clear','Unknown':'Not Reported','Other':'Not Reported','Sleet, Hail (Freezing Rain or Drizzle)':'Rain','Freezing Rain or Drizzle':'Rain'}, inplace = True)
final_dataframe1['Light_Type'].replace({'Dark - Not Lighted':'Dark','Dark - Lighted': 'Dark','Dark - Unknown Lighting':'Dark','Not Reported':'Dark','Other':'Dark','Unknown':'Dark','Dawn':'Dawn/Dusk','Dusk':'Dawn/Dusk'}, inplace = True)
final_dataframe1.dropna(inplace = True)
ohe = OneHotEncoder(handle_unknown = 'ignore')
encoder_df = pd.DataFrame(ohe.fit_transform(final_dataframe1[['Weather']]).toarray())
encoder_df1 = pd.DataFrame(ohe.fit_transform(final_dataframe1[['Light_Type']]).toarray())
encoder_df.rename(columns = {0 : 'Clear', 1 : 'Fog, Smog, Smoke',2: 'Weather Not Reported',3: 'Rain',4: 'Snow'}, inplace = True)
encoder_df1.rename(columns = {0 : 'Dark', 1 : 'Dawn/Dusk',2: 'Daylight'}, inplace = True)
final_df = pd.concat([final_dataframe1.dropna(),encoder_df.dropna(), encoder_df1.dropna()], axis = "columns" )
final_df.dropna(inplace = True)
final_df.to_csv('Test_Details.csv')
    


Crash_Occupant

In [20]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
from collections import defaultdict

def crash_by_occupant(age, sex, seat_pos, injury_sev, from_caseyear, to_caseyear, state, include_occupants, include_nonoccupants):
    url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCrashesByPerson?age={age}&sex={sex}&seatPos={seat_pos}&injurySeverity={injury_sev}&fromCaseYear={from_caseyear}&toCaseYear={to_caseyear}&state={state}&includeOccupants={include_occupants}&includeNonOccupants={include_nonoccupants}&format=json"
    page = requests.get(url)
    # print(page.status_code)
    # soup = BeautifulSoup(page.text,'html.parser')
    # print(soup.prettify())
    data = page.text
    parse_json = json.loads(data)
    # print(type(parse_json))
    # print(parse_json)
    attributeDict = defaultdict(list)
    for attribute in parse_json["Results"][0]:
        attributeDict["State Case"].append(attribute["ST_CASE"])
        attributeDict["Age"].append(attribute["AGE"])
        attributeDict["Sex"].append(attribute["SEX"])
        # attributeDict["Sex_Name"].append(attribute["SEXNAME"])
        # attributeDict["Seat"].append(attribute["SEAT_POS"])
        # attributeDict["Seat_pos"].append(attribute["SEAT_POSNAME"])
        # attributeDict["Fatality"].append(attribute["DEATH_HRNAME"])
        attributeDict["Drinking_Involvement"].append(attribute["DRINKING"])
        attributeDict["Drugs_involvement"].append(attribute["DRUGS"])
        attributeDict["Injury_Severity"].append(attribute["INJ_SEV"])
        attributeDict["Race_Involvement"].append(attribute["RACE"])
        attributeDict["Rollover"].append(attribute["ROLLOVER"])
        attributeDict["Case Year"].append(attribute["CaseYear"])
        # attributeDict["SchoolBus_involvement"].append(attribute["SCH_BUS"])
        # attributeDict["Vehicle_withTrailers"].append(attribute["TOW_VEH"])
        dataframe = pd.DataFrame(attributeDict)
        # dataframe.set_index('CASE_ID')
        # dataframe.sort_index()
        return dataframe
age_upper_limit = 71
age_lower_limit = 18
crash_occupant_df = pd.DataFrame()
for i in range(age_lower_limit, age_upper_limit,1):
    for j in range(1,3,1):
        for k in range(1,5,1):
            crash_occupant_df = crash_occupant_df.append(crash_by_occupant(i,j,11,k,2014,2014,1,True,True))
            # crash_occupant_df.to_csv("April25_set1.csv")
crash_occupant_df['Drinking_Involvement'].replace(to_replace = [8,9],value = [2,2], inplace = True)
crash_occupant_df['Drugs_involvement'].replace(to_replace = [8,9],value = [2,2], inplace = True)
crash_occupant_df['Race_Involvement'].replace(to_replace = [2,3],value = [1,1], inplace = True)
crash_occupant_df['Rollover'].replace(2,1, inplace = True)
crash_occupant_df.to_csv("Occupant_Test.csv")

# crash_by_occupant(18,1,11,2,2014,2014,1,True,True)

<ipython-input-20-1761cf9d6064>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crash_occupant_df = crash_occupant_df.append(crash_by_occupant(i,j,11,k,2014,2014,1,True,True))
<ipython-input-20-1761cf9d6064>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crash_occupant_df = crash_occupant_df.append(crash_by_occupant(i,j,11,k,2014,2014,1,True,True))
<ipython-input-20-1761cf9d6064>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crash_occupant_df = crash_occupant_df.append(crash_by_occupant(i,j,11,k,2014,2014,1,True,True))
<ipython-input-20-1761cf9d6064>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crash_occupant_df = crash_oc